In [ ]:
#Importing necessary libraries
import tensorflow as tf 
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd 
import tensorflow_hub as hub
import os
from tensorflow import keras
from keras.preprocessing import image
from matplotlib.pyplot import imread
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Loading the train dataset
df_train = pd.read_csv('/content/gdrive/MyDrive/train_NoNTTqq.csv')
print(df_train)

            id  ultraviolet_filter  green_filter  red_filter  \
0            1            17.44385      15.71196    16.14848   
1            2            22.02806      24.01481    21.16334   
2            3            23.07242      21.79252    20.51945   
3            4            23.45985      23.41583    20.36645   
4            5            23.89627      23.18005    21.12911   
...        ...                 ...           ...         ...   
134906  134907            19.24538      18.80673    16.41091   
134907  134908            23.41124      22.59072    22.50731   
134908  134909            21.76064      20.16531    20.07795   
134909  134910            18.55473      17.70518    16.67601   
134910  134911            22.07739      20.55631    20.97874   

        near_infrared_filter       alpha      delta  redshift  stellar  
0                  15.647619  158.167937  29.746275  0.094857        1  
1                  20.214615  145.916931  38.083063  0.361631        1  
2           

In [ ]:
#Dropping the id column from the train dataframe
df_train.drop(columns='id', inplace=True)

In [ ]:
import numpy as np

In [ ]:
cf = np.array([[167, 4, 0, 0, 0],
               [2, 144, 4, 1, 5],
               [0, 1, 160, 0, 0],
               [0, 1, 1, 146, 0],
               [0, 1, 3, 0, 170]])

In [ ]:
import seaborn as sns
sns.heatmap(cf, annot = True,fmt ='.4g', cmap ='Blues')

In [ ]:
#Replacing each value of stellar column to be in line with SparseCategoricalCrossentropy
df_train['stellar'].replace({
    1:0,
    2:1,
    3:2
}, inplace=True)

In [ ]:
#Loading the test dataset
df_test = pd.read_csv('/content/gdrive/MyDrive/test_SxgqOdc.csv')
df_test

In [ ]:
#Storing the columns of train dataframe other than stellar in X and storing the stellar column of train dataframe in Y
X = df_train.drop('stellar', axis='columns')
Y = df_train.stellar

In [ ]:
#Train test split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X,Y,test_size=0.00001)

In [ ]:
#Dropping the id colum of test dataframe
df_test_p = df_test.drop('id', axis='columns')
df_test_p

In [ ]:
#Standard Scaling
from sklearn.preprocessing import StandardScaler
st_x= StandardScaler()
x_train_scaled= st_x.fit_transform(x_train)
x_test_scaled= st_x.transform(df_test_p)
x_val_scaled = st_x.transform(x_val)

In [ ]:
#RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=150)
model1.fit(x_train_scaled, y_train) 
model1.score(x_val_scaled, y_val) 

In [ ]:
#Storing the predicted probabilities obtained using RandomForest Classifier for test data
pred_prob_test_rf = np.array(model1.predict_proba(x_test_scaled))
print(pred_prob_test_rf)

In [ ]:
#Adaboost Classifier
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier(n_estimators=150, base_estimator= None,learning_rate=0.1, random_state = 1)
adaboost.fit(x_train_scaled,y_train)

print(adaboost.score(x_val_scaled, y_val))

In [ ]:
#Storing the predicted probabilities obtained using Adaboost Classifier for test data
pred_prob_test_ada = np.array(adaboost.predict_proba(x_test_scaled))
print(pred_prob_test_ada)

In [ ]:
#LGBM Classifier
import lightgbm as ltb
model2 = ltb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model2.fit(x_train_scaled,y_train)
model2.score(x_val_scaled, y_val)

In [ ]:
#Storing the predicted probabilities obtained using LGBM Classifier for test data
pred_prob_test_lgbm = np.array(model2.predict_proba(x_test_scaled))
print(pred_prob_test_lgbm)

In [ ]:
#Installing the catboost library
!pip install catboost

In [ ]:
#CatBoost Classifier
import catboost as cb
model3 = cb.CatBoostClassifier()
model3.fit(x_train_scaled,y_train)
model3.score(x_val_scaled, y_val)

In [ ]:
#Storing the predicted probabilities obtained using CatBoost Classifier for test data
pred_prob_test_cat = np.array(model3.predict_proba(x_test_scaled))
print(pred_prob_test_cat)

In [ ]:
#GradientBoosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, max_features=2, max_depth=2, random_state=0)
gb_clf.fit(x_train_scaled,y_train)
gb_clf.score(x_val_scaled, y_val)

In [ ]:
#Storing the predicted probabilities obtained using GradientBoosting Classifier for test data
pred_prob_test_gb = np.array(gb_clf.predict_proba(x_test_scaled))
print(pred_prob_test_gb)

In [ ]:
#ExtraTrees Classifier
from sklearn.ensemble import ExtraTreesClassifier
etf = ExtraTreesClassifier(n_estimators = 100,criterion ='entropy')
etf.fit(x_train_scaled,y_train)
etf.score(x_val_scaled, y_val)

In [ ]:
#Storing the predicted probabilities obtained using ExtraTrees Classifier for test data
pred_prob_test_etf = np.array(etf.predict_proba(x_test_scaled))
print(pred_prob_test_etf)

In [ ]:
n_epochs_per_model = 40
lr_max = 0.001

In [ ]:
#Learning Rate Scheduling for ANN
import math
def cosine_annealing(epoch):
        cos_inner = (math.pi * (epoch % n_epochs_per_model)) / n_epochs_per_model
        return lr_max / 2 * (math.cos(cos_inner) + 1)

In [ ]:
callback = tf.keras.callbacks.LearningRateScheduler(cosine_annealing)

In [ ]:
#ANN
import tensorflow as tf
from tensorflow import keras


model_ann = keras.Sequential([
    keras.layers.Dense(7, input_shape=(7,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(80, activation='relu'), 
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(3, activation='softmax')
])

model_ann.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

model_ann.fit(x_train_scaled, y_train, callbacks = [callback],validation_data=(x_val_scaled, y_val), epochs= 40)

In [ ]:
#Storing the predicted probabilities obtained using ANN for test data
pred_prob_test_ann = model_ann.predict(x_test_scaled)
print(pred_prob_test_ann)

2811/2811 [==============================] - 5s 2ms/step
[[6.2942110e-02 9.3620700e-01 8.5094135e-04]
 [9.8847586e-01 1.0635775e-02 8.8825234e-04]
 [3.1274137e-01 7.7202404e-03 6.7953837e-01]
 ...
 [9.5337498e-01 4.2527270e-02 4.0977774e-03]
 [2.1943133e-02 9.7703892e-01 1.0179415e-03]
 [9.8134065e-01 1.0600814e-02 8.0584576e-03]]


In [ ]:
#Ensembling with experimental weights
pred_ensem_test_final = (4*pred_prob_test_lgbm + 3*pred_prob_test_rf + 20*pred_prob_test_cat + 2*pred_prob_test_etf + 5*pred_prob_test_gb + pred_prob_test_ada+3*pred_prob_test_ann)/38
print(pred_ensem_test_final)

[[0.11225686 0.87762035 0.0101228 ]
 [0.97486372 0.01585527 0.009281  ]
 [0.25204526 0.02591409 0.72204065]
 ...
 [0.94942116 0.03146894 0.01910991]
 [0.01485498 0.97635912 0.00878591]
 [0.9729171  0.01203826 0.01504464]]


In [ ]:
pred_ensem_test = []
for i in pred_ensem_test_final:
  pred_ensem_test.append(np.argmax(i)+1)
pred_ensem_test = np.array(pred_ensem_test)
print(pred_ensem_test)

[2 1 3 ... 1 2 1]


In [ ]:
#Converting to dataframe
dfx = pd.DataFrame(list(zip(pred_ensem_test)),
               columns = ['stellar'])

In [ ]:
dfx.to_csv('SOS_ensemble_final.csv')